In [10]:
ionex_file = "../jplg3660.12i"

"../jplg3660.12i"

In [ ]:
open(ionex_file) do f
    for i in enumerate(eachline(f))
        print(i[1], ": ", i[2], "\n")
    end
end

In [12]:
pwd()

"/home/antonio/Repos/iono2/julia_scripts"

In [13]:
file = open(ionex_file);
lines = readlines(file)
close(file)

In [ ]:
using Dates
index = 0
numbers = []
z = []
lines_list = []
for line in lines
    index += 1
    if occursin(r"EPOCH OF CURRENT MAP", line)
        date = []
        string = split(line)
        for x in 1:6
            push!(date,parse(Int,string[x]))
        end
        year = date[1]
        month = date[2]
        day = date[3]
        hour = date[4]
        minutes = date[5]
        seconds = date[6]
        #print(DateTime(year, month, day, hour, minutes, seconds), "\n")
    end
    if occursin(r"LAT/LON1/LON2/DLON/H", line)
        #print(index,"\n")
        #push!(lines_list, index)
        #print(index, lines[index], "\n")
        numbers = split(lines[index + 1])
        #for n in numbers
        #    print(typeof(parse(Int,n)))
        #end
        #push!(z, parse(Int,lines[index + 1], "\n")
        #print(lines[index + 2], "\n")
        #print(lines[index + 3], "\n")
        #print(lines[index + 4], "\n")
        #print(lines[index + 5], "\n")
        print('\n',index,'\n')
        for x in 1:5
            numbers = split(lines[index + x])
            for n in numbers
                push!(z, n)
            end
        end
    end
end
        

In [16]:

open("depth.csv","w") do io
    write(io,'x', '\n')
    for i in z
        write(io,i,'\n')
    end
end

In [18]:
using Serialization 
serialize(depth_array, z)

UndefVarError: UndefVarError: depth_array not defined

In [ ]:
using Gadfly
set_default_plot_size(14cm, 8cm)
plot(z=(x,y) -> x*exp(-(x-round(Int, x))^20-y^20),
     xmin=[-180], xmax=[180], ymin=[-90], ymax=[90], Geom.contour)



In [ ]:
p1 = plot([sin,cos], 0, 2pi)
p2 = plot((x,y)->sin(x)+cos(y), 0, 2pi, 0, 2pi)
p3 = spy(ones(33)*sin.(0:(pi/16):2pi)' + cos.(0:(pi/16):2pi)*ones(33)')
hstack(p1)